# Imports

In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

#Statsmodels is a great library we can use to run regressions.
import statsmodels.api as sm
# Seaborn extends the capabilities of Matplotlib
import seaborn as sns
sns.set_style("darkgrid")
# Used for calculating regressions
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

import plotly.express as px
# Allows us to create graph objects for making more customized plots
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

# Dates & Other Constants

In [2]:
PATH = "./Stocks/"

# Start date defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2017-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2021
E_MONTH = 10
E_DAY = 29
E_DATE_STR = "2021-10-29"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

risk_free_rate = 0.0125 # Approximate 10 year bond rate

# Function that Saves Dataframe to CSV

In [3]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

# Function that Returns a Dataframe from a CSV

In [4]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col='Date', 
                         parse_dates=True)
    except FileNotFoundError:
        pass
        # print("File Doesn't Exist")
    else:
        return df

# Add Daily Return to Dataframe

In [5]:
# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    # Save data to a CSV file
    save_dataframe_to_csv(df, ticker)
    return df

# Merge Multiple Stocks in One Dataframe by Column Name

In [6]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        
        # NEW Check if your dataframe has duplicate indexes
        if not df.index.is_unique:
            # Delete duplicates 
            df = df.loc[~df.index.duplicated(), :]
        
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

In [7]:
port_list = ["AMD", "CPRT"]
mult_df = merge_df_by_column_name('daily_return',
                                  '2018-01-02', 
                                  '2021-10-29',
                                  *port_list)
mult_df

,AMD,CPRT
Date,,
2018-01-02,0.068093,0.009493
2018-01-03,0.051913,-0.004816
2018-01-04,0.049351,0.008066
2018-01-05,-0.019802,-0.004801
2018-01-08,0.033670,0.000459
...,...,...
2021-10-25,0.021198,0.006633
2021-10-26,0.004658,-0.000391
2021-10-27,-0.005288,-0.010638


# Calculating Beta

Beta provides the relationship between an investment and the overall market. Risky investments tend to fall further during bad times, but will increase quicker during good times.

Beta is found by dividing the covariance of the stock and the market by the variance of the overall market. It is a measure of systematic risk that can't be diversified away.

$ \beta = \frac{Cov(r_x, r_m)}{\sigma_m^2} $

$ \beta = 0 $ : No relation to market

$ \beta < 1 $ : Less risky than market

$ \beta > 1 $ : More risky than the market
# Examples

Albertsons is a grocery store chain with a low beta of 0.5 because no matter what people need food and pharmecueticals.

AMD manufacturers microchips and is a high beta stock at 1.4 because during hard times there is less demand for their products.


# Get S&P 500 and AMD Data

In [8]:
# # Download data from Yahoo
# sp_df = web.DataReader('^GSPC', 'yahoo', 
#                        '2016-01-01', '2021-10-29')
# sp_df["Date"] = sp_df.index
# sp_df = sp_df[["Date", "Adj Close"]]
# sp_df.to_csv("./Stocks/^GSPC.csv", index=False)

# # Download data from Yahoo
# amd_df = web.DataReader('AMD', 'yahoo', 
#                        '2016-01-01', '2021-10-29')
# amd_df["Date"] = amd_df.index
# amd_df = amd_df[["Date", "Adj Close"]]
# amd_df.to_csv("./Stocks/AMD.csv", index=False)

# add_daily_return_to_df(sp_df, "^GSPC")
# add_daily_return_to_df(amd_df, "AMD")

In [9]:
amd_df = get_df_from_csv('AMD')
amd_df

,Date.1,Adj Close,daily_return
Date,,,
2015-12-31,2015-12-31,2.870000,NaN
2016-01-04,2016-01-04,2.770000,-0.034843
2016-01-05,2016-01-05,2.750000,-0.007220
2016-01-06,2016-01-06,2.510000,-0.087273
2016-01-07,2016-01-07,2.280000,-0.091633
...,...,...,...
2021-10-25,2021-10-25,122.360001,0.021198
2021-10-26,2021-10-26,122.930000,0.004658
2021-10-27,2021-10-27,122.279999,-0.005288


In [10]:
sp_df = get_df_from_csv('^GSPC')
sp_df

,Date.1,Adj Close,daily_return
Date,,,
2015-12-31,2015-12-31,2043.939941,NaN
2016-01-04,2016-01-04,2012.660034,-0.015304
2016-01-05,2016-01-05,2016.709961,0.002012
2016-01-06,2016-01-06,1990.260010,-0.013115
2016-01-07,2016-01-07,1943.089966,-0.023700
...,...,...,...
2021-10-25,2021-10-25,4566.479980,0.004748
2021-10-26,2021-10-26,4574.790039,0.001820
2021-10-27,2021-10-27,4551.680176,-0.005052


# Find Beta for Stock versus S&P

In [11]:
def find_beta(ticker):
    # Tickers analyzed being the S&P and the stock passed
    port_list = ["^GSPC"]
    port_list.append(ticker)
    
    mult_df = merge_df_by_column_name("daily_return", "2016-10-29", "2021-10-29", *port_list)
    
    # Provides the covariance between the securities
    cov = mult_df.cov() * 253
    
    # Get the covariance of the stock and the market
    cov_vs_market = cov.iloc[0,1]
    
    # Get annualized variance of the S&P
    sp_var = mult_df['^GSPC'].var() * 253
    
    # Beta is normally calculated over a 5 year period which is why you may see a difference
    beta = cov_vs_market / sp_var
    return beta

# Get Stock Beta

In [12]:
print("AMD Beta :", find_beta('AMD')) # AMD Beta over 4 Year Data

AMD Beta : 1.4380733338389675



# Capital Asset Pricing Model

Sharpe continued to create the CAPM based on the research of Markowitz. It focuses on investments in stocks and bonds. With it we can more exactly create portfolios that match the risk an investor is willing to assume. CAPM assumes a risk free asset which of course provides a small return. So if the investor wants less risk they simply buy more of the risk free assets.

There is risk that you can limit through diversifaction (Idiosyncratic) and risk that you can't (Systematic). This portfolio contains no Idiosyncratic risk and like before it lies on the efficient frontier.

To find this portfolio we will draw a line ( The Capital Market Line ) from the Y intercept to the efficient frontier.

Here is the formula. The securities expected return equals the risk free asset plus Beta times the market return minus the risk free asset. it is common for $ r_m - r_f $ to be considered 5% which is called the Equity Risk Premium.

$ r_i = r_f + \beta_i (r_m - r_f) $

# Calculate AMDs Expected Return

In [13]:
ri = risk_free_rate + find_beta('AMD') * 0.05
ri

0.08440366669194838

# Sharpe Ratio

William Sharpe created the Sharpe Ratio to find the portfolio that provides the best return for the lowest amount of risk.

Sharpe Ratio = $\frac{r_i - r_f}{\sigma_i}$

$r_f = $ Risk Free Rate

$r_i = $ Rate of Return of the stock

$\sigma_i = $ Standard Deviation of the Stock

As return increases so does the Sharpe Ratio, but as Standard Deviation increase the Sharpe Ratio decreases.

In [14]:
# We can find the Sharpe ratio for AMD
amd_sharpe = (ri - risk_free_rate) / (mult_df['AMD'].std() * 252 ** 0.5)
amd_sharpe

0.1296010512754255

# Get Stock Prices on Date

In [15]:
def get_prices_on_date(stocks_df, date):
    return stocks_df.loc[pd.DatetimeIndex([date])]['Adj Close'].item()

# Returns the Value of Portfolio by Date

In [16]:
def get_port_val_by_date(date, shares, tickers):
    port_prices = merge_df_by_column_name('Adj Close',  date, 
                                  date, *port_list)
    # Convert from dataframe to Python list
    port_prices = port_prices.values.tolist()
    # Trick that converts a list of lists into a single list
    port_prices = sum(port_prices, [])
    
    # Create a list of values by multiplying shares by price
    value_list = []
    for price, share in zip(port_prices, shares):
        value_list.append(price * share)
    
    return sum(value_list)

# Get Value of Portfolio at Beginning and End of Year

In [17]:
port_list = ["GNRC", "CPRT", "ODFL", "AMD", "PAYC", "CHTR",
             "MKC", "PG", "PGR", "NEM", "CCI", "COG"]

port_shares = [25, 20, 22, 26, 1, 1, 4, 1, 5, 28, 3, 7]

# Portfolio value at start of 2020
port_val_start = get_port_val_by_date('2020-01-02', port_shares, port_list)
print("Portfolio Value at Start of January 2020 : $%2.2f" % (port_val_start))

# Portfolio value at end of 2020
port_val_end = get_port_val_by_date('2021-10-29', port_shares, port_list)
print("Portfolio Value at End of October 2021 : $%2.2f" % (port_val_end))

Portfolio Value at Start of January 2020 : $11719.24
Portfolio Value at End of October 2021 : $30568.67


# Calculate Return on Investment

ROI = <img src="https://render.githubusercontent.com/render/math?math=%5Cfrac%7BFinal%20Value%20-%20Initial%20Value%7D%7BInitial%20Value%7D&mode=inline"/>

In [18]:
# Rate of return for portfolio
roi_port = (port_val_end - port_val_start) / port_val_end
print("Portfolio End of October 2021 : %2.2f %%" % (roi_port * 100))

# S&P ROI
sp_df = get_df_from_csv('^GSPC')
sp_val_start = get_prices_on_date(sp_df, '2020-01-02')
sp_val_end = get_prices_on_date(sp_df, '2021-10-29')
sp_roi = (sp_val_end - sp_val_start) / sp_val_end
print("S&P End of October 2021 : %2.2f %%" % (sp_roi * 100))

Portfolio End of October 2021 : 61.66 %
S&P End of October 2021 : 29.26 %


# Find Daily Return for Whole Portfolio

To find the daily return for the whole portfolio, I must multiply the daily price by the number of shares for each security. Then sum those values for all stocks per day. This creates a portfolio list of daily prices. Then I can calculate the daily return.


In [19]:
def get_port_daily_return(sdate, edate, shares, tickers):
    # Merge all daily prices for all stocks into 1 dataframe
    mult_df = merge_df_by_column_name('Adj Close',  sdate, 
                                  edate, *port_list)
    
    # Get the number of stocks in portfolio
    num_cols = len(mult_df.columns)
    
    # Multiply each stock column by the number of shares
    i = 0
    while i < num_cols:
        mult_df[tickers[i]] = mult_df[tickers[i]].apply(lambda x: x * shares[i])
        i += 1
        
    # Create a new column with the sums of all stocks named Total
    mult_df['Total'] = mult_df.iloc[:, 0:num_cols].sum(axis=1)
    
    # Add column for portfolio daily return
    mult_df['daily_return'] = (mult_df['Total'] / mult_df['Total'].shift(1)) - 1
    
    return mult_df

In [20]:
tot_port_df = get_port_daily_return('2020-01-02', '2021-10-29', 
                                    port_shares, port_list)
tot_port_df

,GNRC,CPRT,ODFL,AMD,PAYC,CHTR,MKC,PG,PGR,NEM,CCI,COG,Total,daily_return
Date,,,,,,,,,,,,,,
2020-01-02,2550.250053,1869.600067,2801.878220,1276.599960,271.799988,490.429993,326.336029,117.481445,341.025391,1157.741852,403.065216,113.027060,11719.235273,NaN
2020-01-03,2541.749954,1853.200073,2768.464981,1263.599960,272.709991,494.470001,331.537048,116.691307,340.979500,1147.294830,405.284271,113.355064,11649.336983,-0.005964
2020-01-06,2531.250000,1869.799957,2746.578781,1258.139984,278.089996,497.450012,333.523895,116.853142,347.173347,1158.813278,401.870316,114.273457,11653.816166,0.000385
2020-01-07,2572.500038,1867.400055,2754.457840,1254.500000,279.489990,499.160004,327.796997,116.129654,344.521713,1158.009628,395.071014,114.929440,11683.966373,0.002587
2020-01-08,2500.749969,1880.200043,2767.443466,1243.580048,283.119995,500.769989,326.043884,116.624672,346.557770,1127.204544,396.721069,110.534319,11599.549768,-0.007225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,11752.999878,3065.599976,7143.400269,3181.360016,538.929993,731.010010,322.720001,140.850006,481.749992,1621.479996,532.919998,157.919996,29670.940130,0.005629
2021-10-26,11817.749786,3064.400024,7166.720215,3196.180008,537.190002,728.229980,324.239990,142.850006,481.250000,1615.600021,540.240005,157.289995,29771.940035,0.003404
2021-10-27,12222.000122,3031.799927,7125.580322,3179.279968,531.330017,714.510010,321.880005,141.830002,471.500015,1611.120026,534.750000,154.770004,30040.350418,0.009016


# Find Portfolio Beta

In [21]:
def find_port_beta(port_df, sdate, edate):
    # Will hold data for S&P and my portfolio
    mult_df = pd.DataFrame()
    
    # Mask defining the dates worth of data that we want
    port_mask = (port_df.index >= sdate) & (port_df.index <= edate)
    
    # Get S&P Dataframe
    sp_df = get_df_from_csv('^GSPC')
    
    sp_mask = (sp_df.index >= sdate) & (sp_df.index <= edate)
    
    # Add S&P daily return to dataframe
    mult_df['^GSPC'] = sp_df.loc[sp_mask]['daily_return']
    
    # Add the portfolio daily return data
    mult_df['Portfolio'] = port_df.loc[port_mask]['daily_return']
    
    # Provides the covariance between the securities
    cov = mult_df.cov() * 253
    
    # Get the covariance of the stocks and the market
    cov_vs_market = cov.iloc[0,1]
    
    # Get annualized variance of the S&P
    sp_var = mult_df['^GSPC'].var() * 253
    
    # Beta is normally calculated over a 5 year period which is why you may see a difference
    beta = cov_vs_market / sp_var
    return beta

In [22]:
port_beta = find_port_beta(tot_port_df, '2020-01-02', '2020-12-31')
print("Portfolio Beta : %2.2f %%" % (port_beta * 100))

Portfolio Beta : 91.39 %


# Calculating Alpha¶

Alpha provides a measure of how well a portfolio has performed. The CAPM assumes an Alpha of 0. Good portfolios have a positive Alpha, while poor have negative.

Alpha = R – Rf – beta ( Rm - Rf )

- R represents the portfolio return
- Rf represents the risk-free rate of return
- Beta represents the systematic risk of a portfolio
- Rm represents the market return, per a benchmark



In [23]:
port_alpha = roi_port - risk_free_rate - (port_beta * (sp_roi - risk_free_rate))
print("Portfolio Alpha : %2.2f %%" % (port_alpha * 100))

Portfolio Alpha : 34.81 %
